# Bulk Email Sender

This notebook allows you to send bulk emails using Python's SMTP library.

## Features:
- Send personalized emails to multiple recipients
- Support for HTML templates
- CSV file support for recipient data
- Attachment support
- Email tracking and logging

## Requirements:
Install required packages by running:
```bash
pip install pandas python-dotenv
```

In [ ]:
# Import required libraries
import smtplib
import pandas as pd
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders
import os
from datetime import datetime
import time
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

print("✓ Libraries imported successfully")

## Configuration

Set up your email credentials and SMTP settings. For security, use environment variables or the `.env` file.

In [ ]:
# Email Configuration
SMTP_SERVER = os.getenv('SMTP_SERVER', 'smtp.gmail.com')  # Gmail SMTP server
SMTP_PORT = int(os.getenv('SMTP_PORT', 587))  # TLS port
SENDER_EMAIL = os.getenv('SENDER_EMAIL', 'your_email@gmail.com')
SENDER_PASSWORD = os.getenv('SENDER_PASSWORD', 'your_app_password')
SENDER_NAME = os.getenv('SENDER_NAME', 'Your Name')

print(f"SMTP Server: {SMTP_SERVER}:{SMTP_PORT}")
print(f"Sender: {SENDER_NAME} <{SENDER_EMAIL}>")

## Load Recipients from CSV

Your CSV file should have columns: `email`, `name`, and any other custom fields you want to use in templates.

In [ ]:
# Load recipients from CSV file
def load_recipients(csv_file='recipients.csv'):
    """
    Load recipients from a CSV file.
    Expected columns: email, name (and optional custom fields)
    """
    try:
        df = pd.read_csv(csv_file)
        print(f"✓ Loaded {len(df)} recipients from {csv_file}")
        print(f"\nColumns: {', '.join(df.columns.tolist())}")
        print(f"\nFirst 5 recipients:")
        print(df.head())
        return df
    except FileNotFoundError:
        print(f"✗ Error: File '{csv_file}' not found!")
        return None
    except Exception as e:
        print(f"✗ Error loading CSV: {str(e)}")
        return None

# Load the recipients
recipients_df = load_recipients('recipients.csv')

## Email Template

Define your email subject and body. Use placeholders like `{name}` for personalization.

In [ ]:
# Email subject and body templates
EMAIL_SUBJECT = "Hello {name}!"

EMAIL_BODY_HTML = """
<!DOCTYPE html>
<html>
<head>
    <style>
        body {{ font-family: Arial, sans-serif; line-height: 1.6; color: #333; }}
        .container {{ max-width: 600px; margin: 0 auto; padding: 20px; }}
        .header {{ background-color: #4CAF50; color: white; padding: 20px; text-align: center; }}
        .content {{ padding: 20px; background-color: #f9f9f9; }}
        .footer {{ text-align: center; padding: 10px; font-size: 12px; color: #777; }}
    </style>
</head>
<body>
    <div class="container">
        <div class="header">
            <h1>Welcome!</h1>
        </div>
        <div class="content">
            <p>Dear {name},</p>
            <p>This is a personalized bulk email sent using Python!</p>
            <p>We hope you find this message helpful.</p>
            <p>Best regards,<br>{sender_name}</p>
        </div>
        <div class="footer">
            <p>This email was sent from a bulk email system.</p>
        </div>
    </div>
</body>
</html>
"""

EMAIL_BODY_PLAIN = """
Dear {name},

This is a personalized bulk email sent using Python!

We hope you find this message helpful.

Best regards,
{sender_name}
"""

print("✓ Email templates loaded")

## Email Sending Functions

In [ ]:
def create_email(recipient_email, recipient_name, subject_template, body_html_template, body_plain_template, attachment_path=None):
    """
    Create a MIME email message with personalization.
    """
    # Create message container
    msg = MIMEMultipart('alternative')
    msg['From'] = f"{SENDER_NAME} <{SENDER_EMAIL}>"
    msg['To'] = recipient_email
    msg['Subject'] = subject_template.format(name=recipient_name)
    
    # Create plain text and HTML versions
    plain_text = body_plain_template.format(name=recipient_name, sender_name=SENDER_NAME)
    html_text = body_html_template.format(name=recipient_name, sender_name=SENDER_NAME)
    
    # Attach parts
    part1 = MIMEText(plain_text, 'plain')
    part2 = MIMEText(html_text, 'html')
    msg.attach(part1)
    msg.attach(part2)
    
    # Add attachment if provided
    if attachment_path and os.path.exists(attachment_path):
        with open(attachment_path, 'rb') as attachment:
            part = MIMEBase('application', 'octet-stream')
            part.set_payload(attachment.read())
            encoders.encode_base64(part)
            part.add_header(
                'Content-Disposition',
                f'attachment; filename= {os.path.basename(attachment_path)}'
            )
            msg.attach(part)
    
    return msg


def send_email(msg, recipient_email, server):
    """
    Send an email using the provided SMTP server.
    """
    try:
        server.send_message(msg)
        return True, "Success"
    except Exception as e:
        return False, str(e)


print("✓ Email functions defined")

## Send Bulk Emails

This cell sends emails to all recipients with logging and error handling.

In [ ]:
def send_bulk_emails(recipients_df, subject_template, body_html_template, body_plain_template, 
                     attachment_path=None, delay_seconds=1, test_mode=False):
    """
    Send bulk emails to all recipients in the DataFrame.
    
    Args:
        recipients_df: DataFrame with recipient data
        subject_template: Email subject with placeholders
        body_html_template: HTML email body with placeholders
        body_plain_template: Plain text email body with placeholders
        attachment_path: Optional path to attachment file
        delay_seconds: Delay between emails to avoid spam filters
        test_mode: If True, only send to first recipient
    """
    if recipients_df is None or recipients_df.empty:
        print("✗ No recipients to send emails to!")
        return
    
    # Validate required columns
    if 'email' not in recipients_df.columns or 'name' not in recipients_df.columns:
        print("✗ Error: CSV must have 'email' and 'name' columns!")
        return
    
    # Test mode: only send to first recipient
    if test_mode:
        recipients_df = recipients_df.head(1)
        print("\n🧪 TEST MODE: Sending to first recipient only\n")
    
    # Create log file
    log_filename = f"email_log_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
    log_data = []
    
    # Connect to SMTP server
    try:
        print(f"Connecting to {SMTP_SERVER}:{SMTP_PORT}...")
        server = smtplib.SMTP(SMTP_SERVER, SMTP_PORT)
        server.starttls()
        server.login(SENDER_EMAIL, SENDER_PASSWORD)
        print("✓ Connected to SMTP server\n")
        
        # Send emails
        total = len(recipients_df)
        success_count = 0
        fail_count = 0
        
        print(f"Sending emails to {total} recipient(s)...\n")
        
        for index, row in recipients_df.iterrows():
            recipient_email = row['email']
            recipient_name = row['name']
            
            print(f"[{index + 1}/{total}] Sending to {recipient_name} <{recipient_email}>...", end=' ')
            
            # Create email
            msg = create_email(
                recipient_email, 
                recipient_name, 
                subject_template, 
                body_html_template, 
                body_plain_template,
                attachment_path
            )
            
            # Send email
            success, message = send_email(msg, recipient_email, server)
            
            # Log result
            timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            log_entry = {
                'timestamp': timestamp,
                'email': recipient_email,
                'name': recipient_name,
                'status': 'Success' if success else 'Failed',
                'message': message
            }
            log_data.append(log_entry)
            
            if success:
                print("✓ Sent")
                success_count += 1
            else:
                print(f"✗ Failed: {message}")
                fail_count += 1
            
            # Delay between emails
            if index < total - 1:  # Don't delay after last email
                time.sleep(delay_seconds)
        
        # Close connection
        server.quit()
        print("\n✓ Disconnected from SMTP server")
        
        # Save log
        log_df = pd.DataFrame(log_data)
        log_df.to_csv(log_filename, index=False)
        print(f"\n✓ Log saved to {log_filename}")
        
        # Summary
        print("\n" + "="*50)
        print("SUMMARY")
        print("="*50)
        print(f"Total emails: {total}")
        print(f"✓ Successful: {success_count}")
        print(f"✗ Failed: {fail_count}")
        print("="*50)
        
    except smtplib.SMTPAuthenticationError:
        print("\n✗ Authentication failed! Check your email and password.")
        print("For Gmail, you need to use an 'App Password', not your regular password.")
    except Exception as e:
        print(f"\n✗ Error: {str(e)}")


print("✓ Bulk email function defined")

## Test Email (Send to First Recipient Only)

Run this cell to test your configuration by sending to just the first recipient.

In [ ]:
# Test with first recipient
send_bulk_emails(
    recipients_df=recipients_df,
    subject_template=EMAIL_SUBJECT,
    body_html_template=EMAIL_BODY_HTML,
    body_plain_template=EMAIL_BODY_PLAIN,
    attachment_path=None,  # Set to file path if you want to attach a file
    delay_seconds=1,
    test_mode=True  # Only sends to first recipient
)

## Send to All Recipients

⚠️ **WARNING**: This will send emails to ALL recipients in your CSV file!

Make sure you've tested first and your email content is correct.

In [ ]:
# Send to all recipients
# Uncomment the lines below when you're ready to send

# send_bulk_emails(
#     recipients_df=recipients_df,
#     subject_template=EMAIL_SUBJECT,
#     body_html_template=EMAIL_BODY_HTML,
#     body_plain_template=EMAIL_BODY_PLAIN,
#     attachment_path=None,  # Set to file path if you want to attach a file
#     delay_seconds=2,  # Increase delay to avoid spam filters
#     test_mode=False  # Send to all recipients
# )

print("Uncomment the code above to send emails to all recipients")